# Introduction to Deep Learning: Cours-3

## Multiple Linear Regression

### The Titanic dataset
Consider for example dataset from https://www.kaggle.com/c/titanic/ 

We can use torch `Dataset` class to create a dataset and a test/train split on these data.

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn


class TitanicDataset(Dataset):
    def __init__(self, csv_file):
        raw = pd.read_csv(csv_file)
        
        # Prepare all features
        df = raw.loc[:,["Survived"]].copy()
        df["Pclass"] = raw["Pclass"]
        df["Sex"] = raw["Sex"].apply(lambda s: 0 if s == "male" else 1)
        df["Age"] = raw["Age"].fillna(raw["Age"].median())
        df["SibSp"] = raw["SibSp"]
        df["Parch"] = raw["Parch"]
        df["Fare"] = raw["Fare"].fillna(raw["Fare"].median())
        
        # Store feature names
        self.feature_names = [col for col in df.columns if col != "Survived"]
        
        # Convert to tensors
        self.X = torch.tensor(df[self.feature_names].values, dtype=torch.float64)
        self.y = torch.tensor(df["Survived"].values, dtype=torch.float64)
        
        # Normalize features
        self.X_mean = self.X.mean(dim=0)
        self.X_std = self.X.std(dim=0)
        self.X = (self.X - self.X_mean) / self.X_std
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataset
dataset = TitanicDataset("titanic.csv")
# on colab: dataset = TitanicDataset("todo")

print(f"Features: {dataset.feature_names}")
print(f"Dataset size: {len(dataset)}")
print(f"Sample: {dataset[0]}")

In [ ]:
from torch.utils.data import random_split, DataLoader

# Split into train (80%) and test (20%)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print(f"Total samples: {len(dataset)}")
print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Batch size: {batch_size}")
print(f"Train batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

# Get feature names and normalization params from dataset
feature_names = dataset.feature_names
X_mean = dataset.X_mean
X_std = dataset.X_std

You can try to play with the data. 
For instance here is a plot of the Fare for each age on the train set.

In [ ]:
# Get Age and Fare indices
age_idx = dataset.feature_names.index("Age")
fare_idx = dataset.feature_names.index("Fare")

# Extract full training data for visualization
X_train_full = torch.stack([train_dataset[i][0] for i in range(len(train_dataset))])

# Extract Age and Fare from training set (denormalize for plotting)
age_train = X_train_full[:, age_idx] * X_std[age_idx] + X_mean[age_idx]
fare_train = X_train_full[:, fare_idx] * X_std[fare_idx] + X_mean[fare_idx]

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(age_train, fare_train, alpha=0.5)
plt.xlabel('Age')
plt.ylabel('Fare')
plt.title('Fare vs Age (Training Set)')
plt.grid(True)
plt.show()

There is no clear correlation. 
So let's try something different : try to predict the Fare using all the other features.

In [ ]:
# For linear regression, we predict Fare from all other features
# Get Fare index
fare_idx = dataset.feature_names.index("Fare")

# All features except Fare
linear_feature_names = [f for f in dataset.feature_names if f != "Fare"]
linear_indices = [dataset.feature_names.index(f) for f in linear_feature_names]

print(f"Linear regression features: {linear_feature_names}")
print(f"Target: Fare")
print(f"Number of features: {len(linear_feature_names)}")

### Multiple inputs

Compared to a classic linear regression with one input, one output. We now have multiple inputs.

The simplest possible model is to learn **Linear combination** of the inputs:
$$
\hat{y} = a_{\text{Pclass}} \cdot x_{\text{Pclass}} + a_{\text{Sex}} \cdot x_{\text{Sex}} + a_{\text{Age}} \cdot x_{\text{Age}} + a_{\text{SibSp}} \cdot x_{\text{SibSp}} + a_{\text{Parch}} \cdot x_{\text{Parch}} + b
$$

Or in compact matrix form:
$$
\hat{y} = x^T a + b
$$

where $x \in \mathbb{R}^{5}$ is the feature vector (all features except Fare), $a \in \mathbb{R}^{5}$ is the weight vector, and $b \in \mathbb{R}$ is the bias.

This is a simple generalization of the classic linear regression. Notice that the formula in matrix (vectorized) form is actually really close.

In [ ]:
# Initialize parameters for linear regression
num_linear_features = len(linear_feature_names)
a_linear = torch.zeros(num_linear_features, requires_grad=True, dtype=torch.float64)
b_linear = torch.zeros(1, requires_grad=True, dtype=torch.float64)

# Linear model
def linear_model(X):
    return X @ a_linear + b_linear

We can now train using a gradient descent.

In [ ]:
# MSE loss
loss_fn_linear = nn.MSELoss()

# Training
learning_rate = 0.01  # Lower learning rate for mini-batch
num_epochs = 100
train_losses_linear = []
test_losses_linear = []

for epoch in range(num_epochs):
    # Training phase
    epoch_train_loss = 0
    for X_batch, y_batch in train_loader:
        # Extract features and target
        X_batch_linear = X_batch[:, linear_indices]
        y_batch_linear = X_batch[:, fare_idx]
        
        # Forward pass
        y_pred = linear_model(X_batch_linear)
        loss = loss_fn_linear(y_pred, y_batch_linear)
        
        # Backward pass
        loss.backward()
        
        # Update parameters (gradient descent step per batch)
        with torch.no_grad():
            a_linear -= learning_rate * a_linear.grad
            b_linear -= learning_rate * b_linear.grad
            a_linear.grad.zero_()
            b_linear.grad.zero_()
        
        epoch_train_loss += loss.item() * len(X_batch)
    
    # Average train loss for epoch
    avg_train_loss = epoch_train_loss / len(train_dataset)
    train_losses_linear.append(avg_train_loss)
    
    # Evaluation phase
    epoch_test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch_linear = X_batch[:, linear_indices]
            y_batch_linear = X_batch[:, fare_idx]
            
            y_pred = linear_model(X_batch_linear)
            loss = loss_fn_linear(y_pred, y_batch_linear)
            epoch_test_loss += loss.item() * len(X_batch)
    
    avg_test_loss = epoch_test_loss / len(test_dataset)
    test_losses_linear.append(avg_test_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

print(f"\nFinal train loss: {train_losses_linear[-1]:.4f}")
print(f"Final test loss: {test_losses_linear[-1]:.4f}")
print(f"\nLearned parameters:")
for i, feature in enumerate(linear_feature_names):
    print(f"  a_{feature}: {a_linear[i].item():>7.4f}")
print(f"  b:        {b_linear.item():>7.4f}")

_Note:_ [Last week](2-basics/linear-regression.ipynb), we computed the loss on the entire dataset at each iteration of the learning loop using vectorized operations. This is called **Batched Gradient Descent**.

At the other extreme, it is also possible to compute the gradient after a single data point to update the parameters. This is called **Stochastic Gradient Descent** (see [TP-3](3-autodiff/tp-3-backprop.ipynb)).

Here we use a intermediate solution which computes the loss incrementally minibatch by minibatch, and reset it to 0 at each new epoch. This is called **Minibatch Gradient Descent**

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses_linear, label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Linear Regression Training Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Collect predictions and actuals on test set
y_test_linear_all = []
y_pred_test_linear_all = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch_linear = X_batch[:, linear_indices]
        y_batch_linear = X_batch[:, fare_idx]
        
        y_pred = linear_model(X_batch_linear)
        
        y_test_linear_all.append(y_batch_linear)
        y_pred_test_linear_all.append(y_pred)

y_test_linear = torch.cat(y_test_linear_all)
y_pred_test_linear = torch.cat(y_pred_test_linear_all)

plt.figure(figsize=(10, 6))
plt.scatter(y_test_linear, y_pred_test_linear, alpha=0.5)
plt.plot([y_test_linear.min(), y_test_linear.max()], 
         [y_test_linear.min(), y_test_linear.max()], 
         'r--', label='Perfect prediction')
plt.xlabel('Actual Fare (normalized)')
plt.ylabel('Predicted Fare (normalized)')
plt.title(f'Linear Regression: Predicted vs Actual Fare (Test Set)')
plt.legend()
plt.grid(True)
plt.show()

## Logistic Regression

Now the original question for this dataset is: predict whether a passenger will survive the sinking of the Titanic based on data such as their age, sex, or ticket price.

Compared to the **Linear regression** we now want to predict a **class**.

- The input can be more than just one number (multiple features).
- The output is a class, not a real number.

### The Logistic Regression Model

Logistic regression is similar to linear regression but for classification problems.
The model has two steps:

1. **Linear combination** of all the features:
$$
l = a_{\text{Pclass}} \cdot x_{\text{Pclass}} + a_{\text{Sex}} \cdot x_{\text{Sex}} + a_{\text{Age}} \cdot x_{\text{Age}} + a_{\text{SibSp}} \cdot x_{\text{SibSp}} + a_{\text{Parch}} \cdot x_{\text{Parch}} + a_{\text{Fare}} \cdot x_{\text{Fare}} + b
$$

Features:
- **Pclass**: passenger class (1st, 2nd, or 3rd)
- **Sex**: gender (0=male, 1=female)
- **Age**: age in years
- **SibSp**: number of siblings/spouses aboard
- **Parch**: number of parents/children aboard
- **Fare**: ticket price

Or in compact matrix form:
$$
l = x^T a + b
$$

where $x \in \mathbb{R}^{6}$ is the feature vector, $a \in \mathbb{R}^{6}$ is the weight vector, and $b \in \mathbb{R}$ is the bias.


2. **Sigmoid activation** to get a probability of survival:
$$
\hat{y} = \sigma(l) = \frac{1}{1 + e^{-l}}
$$

The sigmoid function $\sigma$ maps any real number to a value between 0 and 1. We interpret $\hat{y}$ as the probability of survival.

For prediction, we classify as survived if $\hat{y} > 0.5$, and not survived otherwise.

In [ ]:
import matplotlib.pyplot as plt

# Create range of values
l = torch.linspace(-10, 10, 200)
sigma = F.sigmoid(l)

plt.figure(figsize=(10, 6))
plt.plot(l, sigma, linewidth=2, label=r'$\sigma(l) = \frac{1}{1+e^{-l}}$')
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='Decision boundary')
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.xlabel('l (linear combination)', fontsize=12)
plt.ylabel(r'$\sigma(l)$ (probability)', fontsize=12)
plt.title('Sigmoid Function', fontsize=14)
plt.legend(fontsize=11)
plt.ylim(-0.1, 1.1)
plt.show()

### Model Implementation

In [ ]:
# Initialize parameters
num_features = len(feature_names)
a = torch.zeros(num_features, requires_grad=True, dtype=torch.float64)  # weights for all features
b = torch.zeros(1, requires_grad=True, dtype=torch.float64)  # bias

# Logistic regression model
def model(X):
    l = X @ a + b  # linear combination
    return F.sigmoid(l)  # apply sigmoid

print(f"Number of features: {num_features}")
print(f"Features: {feature_names}")

### Loss Function

Our model predicts $\hat{y}_i$, the probability that passenger $i$ survives. For a single observation:
- If $y_i = 1$ (survived), the likelihood of this outcome is $\hat{y}_i$
- If $y_i = 0$ (died), the likelihood of this outcome is $1-\hat{y}_i$

We can write both cases compactly as: $P(y_i) = \hat{y}_i^{y_i} (1-\hat{y}_i)^{1-y_i}$

To find the best parameters, we want to **maximize the likelihood** of observing our data. For all $n$ observations (assuming independence):

$$
L = \prod_{i=1}^{n} \hat{y}_i^{y_i} (1-\hat{y}_i)^{1-y_i}
$$

Taking the logarithm (which preserves the maximum):

$$
\log L = \sum_{i=1}^{n} \left[ y_i \log(\hat{y}_i) + (1-y_i) \log(1-\hat{y}_i) \right]
$$

We just negate and average to get our loss function to minimize.

This is called the **[Binary Cross-Entropy (BCE) loss](https://docs.pytorch.org/docs/stable/generated/torch.nn.BCELoss.html)**:

$$
\mathcal{L} = -\frac{1}{n}\sum_{i=1}^{n} \left[ y_i \log(\hat{y}_i) + (1-y_i) \log(1-\hat{y}_i) \right]
$$

In [ ]:
# Binary cross-entropy loss using PyTorch's built-in
loss_fn = torch.nn.BCELoss()

### Training with Gradient Descent

We use our usual gradient descent to optimize the parameters using PyTorch's autograd.

In [ ]:
# Training parameters
learning_rate = 0.01  # Lower learning rate for mini-batch
num_epochs = 100

# Track loss history
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    # Training phase
    epoch_train_loss = 0
    for X_batch, y_batch in train_loader:
        # Forward pass
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        
        # Backward pass
        loss.backward()
        
        # Update parameters (gradient descent step per batch)
        with torch.no_grad():
            a -= learning_rate * a.grad
            b -= learning_rate * b.grad
            a.grad.zero_()
            b.grad.zero_()
        
        epoch_train_loss += loss.item() * len(X_batch)
    
    # Average train loss for epoch
    avg_train_loss = epoch_train_loss / len(train_dataset)
    train_losses.append(avg_train_loss)
    
    # Evaluation phase
    epoch_test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            epoch_test_loss += loss.item() * len(X_batch)
    
    avg_test_loss = epoch_test_loss / len(test_dataset)
    test_losses.append(avg_test_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

print(f"\nFinal train loss: {train_losses[-1]:.4f}")
print(f"Final test loss: {test_losses[-1]:.4f}")

### Visualize Training Progress

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss Over Time')
plt.legend()
plt.grid(True)
plt.show()

### Model Evaluation

In [ ]:
# Evaluate on training set
train_correct = 0
train_total = 0
with torch.no_grad():
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch)
        y_pred_class = (y_pred > 0.5).long()
        train_correct += (y_pred_class == y_batch).sum().item()
        train_total += len(y_batch)

train_accuracy = train_correct / train_total

# Evaluate on test set
test_correct = 0
test_total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        y_pred_class = (y_pred > 0.5).long()
        test_correct += (y_pred_class == y_batch).sum().item()
        test_total += len(y_batch)

test_accuracy = test_correct / test_total

print(f"Train accuracy: {train_accuracy:.2%}")
print(f"Test accuracy: {test_accuracy:.2%}")

# Print learned parameters
print(f"\nLearned parameters:")
for i, feature in enumerate(feature_names):
    print(f"  a_{feature}: {a[i].item():>7.4f}")
print(f"  b:        {b.item():>7.4f}")

### Baseline: Simple Rule-Based Model

Let's compare our logistic regression to a simple rule: "all women survive, all men die"

In [ ]:
# Simple rule: women survive (1), men die (0)
# Get the Sex feature index
sex_idx = feature_names.index("Sex")

# Apply rule on test set
test_correct_simple = 0
test_total_simple = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_pred_simple = (X_batch[:, sex_idx] > 0).long()  # normalized sex > 0 means female
        test_correct_simple += (y_pred_simple == y_batch).sum().item()
        test_total_simple += len(y_batch)

test_accuracy_simple = test_correct_simple / test_total_simple

print(f"Simple rule test accuracy: {test_accuracy_simple:.2%}")
print(f"Logistic regression test accuracy: {test_accuracy:.2%}")
print(f"\nImprovement: {(test_accuracy - test_accuracy_simple)*100:.2f} percentage points")

### Predict Your Own Survival

Use the trained model to estimate your survival probability:

In [ ]:
# Enter your information here
my_pclass = 1        # ticket class (1, 2, or 3)
my_sex = 1           # 0 for male, 1 for female
my_age = 30          # your age in years
my_sibsp = 0         # number of siblings/spouses aboard
my_parch = 0         # number of parents/children aboard
my_fare = 50.0       # ticket price (in 1912 pounds, typical range: 7-500)

# Create feature vector
my_features_dict = {
    "Pclass": my_pclass,
    "Sex": my_sex,
    "Age": my_age,
    "SibSp": my_sibsp,
    "Parch": my_parch,
    "Fare": my_fare
}

# Normalize using the same statistics as training data
my_features_list = []
for i, feature in enumerate(feature_names):
    value = my_features_dict[feature]
    norm_value = (value - X_mean[i].item()) / X_std[i].item()
    my_features_list.append(norm_value)

# Create feature tensor
my_features = torch.tensor(my_features_list, dtype=torch.float64)

# Predict survival probability
with torch.no_grad():
    my_survival_prob = model(my_features).item()
    
print(f"Your profile:")
print(f"  Pclass: {my_pclass}")
print(f"  Sex: {'Female' if my_sex == 1 else 'Male'}")
print(f"  Age: {my_age} years")
print(f"  SibSp: {my_sibsp}")
print(f"  Parch: {my_parch}")
print(f"  Fare: £{my_fare:.2f}")
print(f"\nEstimated survival probability: {my_survival_prob:.1%}")
print(f"Prediction: {'Would survive' if my_survival_prob > 0.5 else 'Would not survive'}")

## Logistic Regression with PyTorch's nn.Module

Now let's implement the same logistic regression using PyTorch's neural network library.

In [ ]:
class LogisticRegressionNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1, dtype=torch.float64)
    
    def forward(self, x):
        return F.sigmoid(self.linear(x)).squeeze()

# Create model
model_nn = LogisticRegressionNN(num_features)
print(f"Model: {model_nn}")
print(f"\nInitial parameters:")
print(f"  Weight: {model_nn.linear.weight.data}")
print(f"  Bias: {model_nn.linear.bias.data}")

In [ ]:
# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model_nn.parameters(), lr=0.01)

# Training
num_epochs_nn = 100
train_losses_nn = []
test_losses_nn = []

for epoch in range(num_epochs_nn):
    # Training phase
    model_nn.train()
    epoch_train_loss = 0
    for X_batch, y_batch in train_loader:
        # Forward pass
        y_pred = model_nn(X_batch)
        loss = criterion(y_pred, y_batch)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_train_loss += loss.item() * len(X_batch)
    
    avg_train_loss = epoch_train_loss / len(train_dataset)
    train_losses_nn.append(avg_train_loss)
    
    # Evaluation phase
    model_nn.eval()
    epoch_test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model_nn(X_batch)
            loss = criterion(y_pred, y_batch)
            epoch_test_loss += loss.item() * len(X_batch)
    
    avg_test_loss = epoch_test_loss / len(test_dataset)
    test_losses_nn.append(avg_test_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs_nn}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

print(f"\nFinal train loss: {train_losses_nn[-1]:.4f}")
print(f"Final test loss: {test_losses_nn[-1]:.4f}")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses_nn, label='Train Loss')
plt.plot(test_losses_nn, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss Over Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Evaluate nn.Module model
model_nn.eval()
train_correct_nn = 0
train_total_nn = 0
with torch.no_grad():
    for X_batch, y_batch in train_loader:
        y_pred = model_nn(X_batch)
        y_pred_class = (y_pred > 0.5).long()
        train_correct_nn += (y_pred_class == y_batch).sum().item()
        train_total_nn += len(y_batch)

test_correct_nn = 0
test_total_nn = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_pred = model_nn(X_batch)
        y_pred_class = (y_pred > 0.5).long()
        test_correct_nn += (y_pred_class == y_batch).sum().item()
        test_total_nn += len(y_batch)

train_accuracy_nn = train_correct_nn / train_total_nn
test_accuracy_nn = test_correct_nn / test_total_nn

print(f"Train accuracy: {train_accuracy_nn:.2%}")
print(f"Test accuracy: {test_accuracy_nn:.2%}")

print(f"\nLearned parameters (nn.Module):")
print(f"  Weight: {model_nn.linear.weight.data.squeeze()}")
print(f"  Bias: {model_nn.linear.bias.data}")